In [1]:
# default_exp datapipe
from nbdev import *

# Photos GPS - extract GPS data from photos

* Required as Apple Watch stopped recording GPS data because of a bug :(
* AppleWatch (7.0) and iPhone (14.0) update issues
* Impacted Northern Beaches walks

## TODO

* Need to sort the meta-data and add this walk data to the walk database and also to the cache file

In [2]:
#export
import os
import pandas as pd
from dateutil.parser import parse
import datetime as dt
import sqlite3 as sql
from pathlib import Path
import tomli
from GPSPhoto import gpsphoto # requires GPSPhoto, exifread & piexif libraries
import folium


In [3]:
WALK_DETAILS_FILE = 'walk_details.toml'
walk_details = Path('../' + WALK_DETAILS_FILE)


In [4]:
walk_details

Path('../walk_details.toml')

In [5]:
with open(walk_details, encoding="utf-8") as f:
    walk_details_dict = tomli.load(f)

In [6]:
walk_details_dict

{'walks': [{'short_name': 'B2M', 'name': 'Bondi to Manly'},
  {'short_name': 'B2W', 'name': 'Bondi to Wollongong'},
  {'short_name': 'D2C', 'name': 'Drummoyne to Cockatoo'},
  {'short_name': 'GNW', 'name': 'Great North Walk'},
  {'short_name': 'GTL', 'name': 'Gladesville Loop'},
  {'short_name': 'GNW', 'name': 'Great North Walk'},
  {'short_name': 'GWW', 'name': 'Great West Walk', 'status': 'incomplete'},
  {'short_name': 'OLD', 'name': 'Old Bar'},
  {'short_name': 'STM', 'name': "St Michael's Golf Course"},
  {'short_name': 'SNM', 'name': 'Snowy Mountains (Thredo)'},
  {'short_name': 'WNG',
   'name': 'Newcastle to Sydney',
   'status': 'incomplete'}]}

In [7]:
def add_walk_photo_data_for_app(db_file, n_rows_used=5):
    # read in all of the walks data and sample at an appropriate frequency and cache for faster use in the app
    db_conn = sql.connect(db_file)
    walk_df = pd.read_sql_query('SELECT * FROM walks', db_conn)

    UNUSED_COLUMNS = ['dist', 'speed']

    walk_df.drop(UNUSED_COLUMNS, axis=1, inplace=True)
    walk_df.dropna(inplace=True)      # TODO: Check why there are a few NaNs
    walk_df = walk_df.iloc[::n_rows_used].reset_index()    # downsample

    walk_df.to_feather(Path(db_file.as_posix().replace('.db', '.cache.feather')))
    
    return walk_df

In [8]:
# walk_df = create_walk_datafile_for_app(db_file, 10)

In [9]:
# walk_df[walk_df['lat'].isna()]

In [10]:
# Path(db_file.as_posix().replace('.db', '.cache.feather'))

In [11]:
PHOTO_DIR1 = r'/Users/mjboothaus/icloud/Data/HealthFit/FIT/B2M/M2P/Queenscliff & North Curl Curl, 21 September 2020/'

In [12]:
PHOTO_DIR2 = r'/Users/mjboothaus/icloud/Data/HealthFit/FIT/B2M/M2P/Newport, 28 September 2020/'
PHOTO_DIR3 = r'/Users/mjboothaus/icloud/Data/HealthFit/FIT/B2M/M2P/19 October 2020/'

In [13]:
photo_details = ['walk_date', 'dir']   # 19 October 2020 - Palm Beach leg

In [14]:
LEN_FILENAME_1 = 23
LEN_FILENAME_2 = 20
LEN_FILENAME_3 = 21

In [15]:
def extract_gps_data_from_photos(LEN_FILENAME, PHOTO_DIR):
    if Path(PHOTO_DIR).exists():
        photo_files = [file for file in os.listdir(PHOTO_DIR) if file.endswith('.jpeg')]
        photo_files.sort(key=lambda x : int(x[LEN_FILENAME:].replace('.jpeg', '')))
        photo_GPS = []
        for iFile, file in enumerate(photo_files):
            gps_data = gpsphoto.getGPSData(PHOTO_DIR + file)
            try:
                photo_GPS.append((gps_data['Latitude'], gps_data['Longitude']))
            except:
                pass
    return photo_GPS

In [16]:
walk1_points = extract_gps_data_from_photos(LEN_FILENAME_1, PHOTO_DIR1)

In [17]:
walk1_df = pd.DataFrame(walk1_points, columns=['lat', 'lon'])

In [18]:
walk1_df.head()

,lat,lon
0,-33.786153,151.287889
1,-33.786142,151.287994
2,-33.786175,151.289397
3,-33.786214,151.290772
4,-33.786206,151.290803


In [19]:
walk2_points = extract_gps_data_from_photos(LEN_FILENAME_2, PHOTO_DIR2)

In [20]:
walk2_df = pd.DataFrame(walk2_points, columns=['lat', 'lon'])

In [21]:
walk2_df.head()

,lat,lon
0,-33.738578,151.303894
1,-33.738358,151.306458
2,-33.738278,151.306733
3,-33.735492,151.305831
4,-33.734639,151.305375


In [22]:
walk3_points = extract_gps_data_from_photos(21, PHOTO_DIR3)
walk3_df = pd.DataFrame(walk3_points, columns=['lat', 'lon'])

In [23]:
walk3_df.head()

,lat,lon
0,-33.654347,151.321625
1,-33.654067,151.322922
2,-33.653478,151.323258
3,-33.652261,151.323853
4,-33.649722,151.325042


In [24]:
start_coord = [-33.9, 151.3]

In [25]:
map_handle = folium.Map(start_coord, zoom_start=13, detect_retina=True, control_scale=True)

In [26]:
def plot_walk_points(walk_points, map_handle, linecolour, linewidth):
    folium.PolyLine(walk_points, color=linecolour, weight=linewidth).add_to(map_handle)

In [27]:
plot_walk_points(walk1_points, map_handle, 'blue', 6)

In [28]:
plot_walk_points(walk2_points, map_handle, 'red', 6)

In [29]:
plot_walk_points(walk3_points, map_handle, 'blue', 6)

In [30]:
map_handle.fit_bounds(map_handle.get_bounds())

In [31]:
map_handle